In [0]:
from pyspark.sql.functions import year,month,dayofmonth,col,dayofweek,to_timestamp,from_unixtime,unix_timestamp,expr

storage_account_name = "tarifihicham1cs"
storage_account_access_key = "Ztcio+a8JNZrV2mX9Gjl73jVSsvU1AVPVztAx0b7eYuZMwwo7u3y6Ko0A1Kjp8/MkgWXuAmN8QS0+ASt722nhA=="
container_name = "tarifihichamcontainer"

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
  storage_account_access_key)

original_path = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/public_transport_data/raw")

for path in original_path:
    filepath = dbutils.fs.ls(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/public_transport_data/raw/"+path.name)
    for filename in filepath:
        if filename.name.endswith(".csv"):
            df = spark.read.format("csv")\
                .option("header", "true")\
                .load(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/public_transport_data/raw/{path.name}/{filename.name}")
            # Extract year
            df = df.withColumn("Year", year(df["Date"]))
            # Extract month
            df = df.withColumn("Month", month(df["Date"]))
            # Extract day
            df = df.withColumn("Day", dayofmonth(df["Date"]))
            # Extract day of week
            df = df.withColumn("DayOfWeek", dayofweek(df["Date"]))
            # Extract duration Calculate the duration in minutes
            df = df.withColumn("Duration", expr("(unix_timestamp(ArrivalTime, 'HH:mm') - unix_timestamp(DepartureTime, 'HH:mm')) / 60"))
            
            display(df)
            
